In [2]:
import os
with open(".env","r") as f:
    key = f.read().strip()
os.environ["OPENAI_API_KEY"] = key

## Interacting with OpenAI programmatically 
* Set up a client
* Create a prompt
* To provide a context for the prompt, use a message object
* Specify the openAI model you can plan on using
* Make an API Call

In [ ]:
# Client
import openai
from openai import OpenAI
client = OpenAI()

# Prompt
 
prompt = "Who holds the fastest 100 metres record?"

#Message
#Let's say we onlt want the record for the Olympics
#We'll use the system attribute to set the context

context = "Your answers should be confined to the summer olympics"
message =[
    {"role": "system", "content": context},
    {"role": "user", "content": prompt}
]

# Specify the model
model = "gpt-4o"

# Call the API
response = client.chat.completions.create (
    #Temperature determines how creative the response should be, if you want creative answers set temperature to more than 0
    model = model,
    messages = message,
    temperature=0
)

# Preety print the response
import pprint
pprint.pprint(response.choices[0].message.content)


In [ ]:
response

In [ ]:
def get_completion(message, client=client, model="gpt-4o", temperature=0):
    message=message
    response = client.chat.completions.create(
        model = model,
        messages = message,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
prompt = """I have data that consists of:
* 10 independent features, all normally distributed
* one dependent variable with binary 0 or 1 values
Generate sample data (100 samples) and write python code 
using the sklearn library for 3 different prediction models. 
Report their AUC scores"""

message=[
    {"role": "system", "content":"You are a machine learning expert"},
    {"role": "user", "content": prompt}
]

response = get_completion(message=message)
print(response)

- Ask a follow up question
* Set the context by including the previous prompt and response
* Then ask the question

In [ ]:
prompt_2 = """Thank you. That was very helpful. For the same problem, could you create python code
for a simple neural network model using tensorflow and keras?"""

message = [
    {"role": "system", "content": "You are a machine learning expert"},
    {"role": "user", "content": prompt},
    {"role": "system", "content": response},
    {"role": "user", "content": prompt_2}
]
print(get_completion(message=message))

The fastest 100m example again

In [ ]:
prompt = "Who holds the fastest 100 metres record, the fastest 200m record and the 400m record?"
message = [
    {"role": "system", "content": "You are an athletics expert"},
    {"role": "user", "content": prompt}
]
pprint.pprint(get_completion(message=message))

In [ ]:
prompt_2 = "Only for the Olympics"

message = [
    {"role": "system", "content": "You are an athletics expert"},
    {"role": "user", "content": prompt},
    {"role": "system", "content": response},
    {"role": "user", "content": prompt_2}
]
pprint.pprint(get_completion(message=message))